In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import copy 
from keras.callbacks import EarlyStopping

tf.config.run_functions_eagerly(True)

In [3]:
df=pd.read_csv('Transitions.csv')
df['transition']=df['transition'].apply(lambda x: 0 if x=="['OFFLINE_SEARCH_PAGE', 'Playing_window']" else x)
df['transition']=df['transition'].apply(lambda x: 1 if x=="['Playing_window', 'BACK_PRESSED_PAGE']" else x)
df['transition']=df['transition'].apply(lambda x: 2 if x=="['Playlist_inside', 'Playing_window']" else x)

In [4]:
sorted_df = df.sort_values(by='event_timestamp')

In [5]:
df=sorted_df

In [6]:
df=df.reset_index(drop=True)

In [7]:
import numpy as np



class ContextualBandits:
    def __init__(self, d, alpha):
        self.d = d  # dimension
        self.alpha = alpha  # exploration parameter
        self.A = {}  # dictionary to store A matrices for each arm
        self.b = {}  # dictionary to store b vectors for each arm
        self.c= {}
        self.rewarddd = {}
        
    
    def create_one_hot_vector(self,index, length):
        vector = np.zeros(length)
        vector[index] = 1
        return vector
    
    def create_vector(self,n, d):
        vector = np.full((d, 1), 0.2)
        vector[n] = 1
        return vector

    def choose_arm(self, features,day):
        p_values = {}  # dictionary to store p values for each arm

        for arm,z in features.items():
            if arm not in self.A:  # if arm is new, initialize A and b
                self.A[arm] = np.identity(self.d)
                #self.A[arm] = np.eye(d) + 0.1 * np.ones((d, d))
                self.b[arm] = np.zeros((self.d, 1))
                self.c[arm] = np.zeros((self.d, 1))
                self.rewarddd[arm] = np.zeros((self.d, 1))
            
            x = self.create_vector(day,7)
            
            A_inv = np.linalg.inv(self.A[arm])
            theta_hat = np.dot(A_inv, self.b[arm])
            p_values[arm] = np.dot(theta_hat.T, x) + self.alpha * np.sqrt(np.dot(np.dot(x.T, A_inv), x))

        chosen_arm = max(p_values, key=p_values.get)
        self.c[chosen_arm][day]+=1
        return chosen_arm

    def update_params(self, arm, day, reward):
        x = self.create_vector(day,7)
        x=x.reshape(1,-1)
        x_T = x.reshape((-1, 1))
        self.A[arm] += np.dot(x_T,x)
        self.b[arm] += reward * x_T

In [8]:
import random
import time

d = 7  # dimension
alpha = 0.4  # exploration parameter
bandits = ContextualBandits(d, alpha)
rewardd=0

T = 1000 # number of iterations
features = {0: np.array([0, 1, 2, 3, 4, 5, 6]), 1: np.array([0,1,2,3,4,5,6]), 2: np.array([0,1,2,3,4,5,6])}  
for t in range(1, T+1):
    day = random.randint(0, 6)
    chosen_arm = bandits.choose_arm(features,day)
    reward = df.loc[(df['transition'] == chosen_arm) & (df['days']==day), 'ad_clicked'].sample().values[0]
    bandits.update_params(chosen_arm, day, reward)
    bandits.rewarddd[chosen_arm][day]+=reward

In [10]:
import random
import time

d = 7  # dimension
alpha = 0.4  # exploration parameter
bandits = ContextualBandits(d, alpha)
rewardd=0

T = 1000 # number of iterations
features = {0: np.array([0, 1, 2, 3, 4, 5, 6]), 1: np.array([0,1,2,3,4,5,6]), 2: np.array([0,1,2,3,4,5,6])}  
for t in range(1, T+1):
    day = random.randint(0, 6)
    chosen_arm = bandits.choose_arm(features,day)
    reward = df.loc[(df['transition'] == chosen_arm) & (df['days']==day), 'ad_clicked'].values[int(bandits.c[chosen_arm][day])]
    bandits.update_params(chosen_arm, day, reward)
    bandits.rewarddd[chosen_arm][day]+=reward

In [11]:
tmp = df[['transition','ad_clicked']].copy()

In [12]:
# print("Current time:", time.strftime("%H:%M:%S"))    
start_time = time.time()
for t in range(1,100):
    #reward = df[['transition'],['ad_clicked']].loc[df['transition']==chosen_arm].sample()['ad_clicked'].values[0] 
#     tmp = df.sample(1000)
    reward = tmp.loc[tmp['transition'] == chosen_arm, 'ad_clicked'].sample().values[0]
    # print("Current time:", time.strftime("%H:%M:%S"))
print(time.time()-start_time)

1.699350118637085


In [13]:
# print("Current time:", time.strftime("%H:%M:%S"))    
start_time = time.time()
for t in range(1,100):
    #reward = df[['transition'],['ad_clicked']].loc[df['transition']==chosen_arm].sample()['ad_clicked'].values[0] 
    
    reward = df.loc[df['transition'] == chosen_arm, 'ad_clicked'].sample().values[0]
    # print("Current time:", time.strftime("%H:%M:%S"))
print(time.time()-start_time)

1.6734638214111328


In [14]:
df

,user_pseudo_id,event_date,event_timestamp,firebase_screen,page_2,ga_session_number,ad_action_type,ga_session_id,page_1,transition,ad_clicked,days
0,b7aeb2e70454f9f97f2b56297be9d34a,20230607,1686076202169004,Splash_screen,Playing_window,843,AD_CLOSED,1686076119,Playlist_inside,2,0,5
1,46e67e7650f34cf764246114f8afc972,20230607,1686076202566000,Playing_window,BACK_PRESSED_PAGE,2392,AD_OPENED,1686076153,Playing_window,1,0,5
2,4d2185131e8ea1d6b51bddefc38c67ce,20230607,1686076203460001,Playlist_inside,Playing_window,223,AD_OPENED,1686076166,Playlist_inside,2,0,5
3,4eaa100130d6bb743b1819de5d51dbe9,20230607,1686076210244010,Playing_window,BACK_PRESSED_PAGE,535,AD_OPENED,1686076197,Playing_window,1,0,5
4,398996510d8f417cf19c59d877091899,20230607,1686076213280007,Search_common,Playing_window,36,AD_OPENED,1686076199,OFFLINE_SEARCH_PAGE,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2069916,fd8665f6ee4a9d456a3ad3b747dfa25e,20230615,1686855575806106,Playlist_inside,Playing_window,2110,AD_OPENED,1686852512,Playlist_inside,2,0,6
2069917,fd8665f6ee4a9d456a3ad3b747dfa25e,20230615,1686855575806106,Playlist_inside,Playing_window,2110,AD_OPENED,1686852512,Playlist_inside,2,0,6
2069918,fd8665f6ee4a9d456a3ad3b747dfa25e,20230615,1686855575806106,Playlist_inside,Playing_window,2110,AD_OPENED,1686852512,Playlist_inside,2,0,6
2069919,fd8665f6ee4a9d456a3ad3b747dfa25e,20230615,1686855575806106,Playlist_inside,Playing_window,2110,AD_OPENED,1686852512,Playlist_inside,2,0,6


In [15]:
bandits.A

{0: array([[64.44, 28.56, 30.16, 26.96, 27.92, 27.44, 26.96],
        [28.56, 58.68, 29.2 , 26.  , 26.96, 26.48, 26.  ],
        [30.16, 29.2 , 68.28, 27.6 , 28.56, 28.08, 27.6 ],
        [26.96, 26.  , 27.6 , 49.08, 25.36, 24.88, 24.4 ],
        [27.92, 26.96, 28.56, 25.36, 54.84, 25.84, 25.36],
        [27.44, 26.48, 28.08, 24.88, 25.84, 51.96, 24.88],
        [26.96, 26.  , 27.6 , 24.4 , 25.36, 24.88, 49.08]]),
 1: array([[65.6 , 29.88, 31.32, 32.44, 29.24, 28.6 , 28.12],
        [29.88, 60.8 , 30.52, 31.64, 28.44, 27.8 , 27.32],
        [31.32, 30.52, 69.44, 33.08, 29.88, 29.24, 28.76],
        [32.44, 31.64, 33.08, 76.16, 31.  , 30.36, 29.88],
        [29.24, 28.44, 29.88, 31.  , 56.96, 27.16, 26.68],
        [28.6 , 27.8 , 29.24, 30.36, 27.16, 53.12, 26.04],
        [28.12, 27.32, 28.76, 29.88, 26.68, 26.04, 50.24]]),
 2: array([[65.6 , 29.72, 31.32, 28.12, 29.24, 28.6 , 32.6 ],
        [29.72, 59.84, 30.36, 27.16, 28.28, 27.64, 31.64],
        [31.32, 30.36, 69.44, 28.76, 29.88,

In [16]:
bandits.b

{0: array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 1: array([[0.2],
        [0.2],
        [0.2],
        [1. ],
        [0.2],
        [0.2],
        [0.2]]),
 2: array([[0.2],
        [0.2],
        [0.2],
        [0.2],
        [0.2],
        [0.2],
        [1. ]])}

In [17]:
rewardd

0

In [18]:
bandits.c

{0: array([[53.],
        [47.],
        [57.],
        [37.],
        [43.],
        [40.],
        [37.]]),
 1: array([[53.],
        [48.],
        [57.],
        [64.],
        [44.],
        [40.],
        [37.]]),
 2: array([[53.],
        [47.],
        [57.],
        [37.],
        [44.],
        [40.],
        [65.]])}

In [19]:
bandits.rewarddd

{0: array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]),
 1: array([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.]]),
 2: array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]])}